In [17]:
import torch
import torch.nn as nn
import torchvision.models as models

def get_pretrained_regression_model(output_size):
    # Load the pretrained ResNet18 model
    pretrained_model = models.resnet18(weights='DEFAULT')
    
    # Modify the last fully connected layer for regression with custom output size
    in_features = pretrained_model.fc.in_features
    pretrained_model.fc = nn.Linear(in_features, output_size)
    
    return pretrained_model

model = get_pretrained_regression_model(100)

In [11]:
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA 
from torchvision import transforms
from PIL import Image
import numpy as np

# Define an imaginary dataset class
class ImaginaryDataset(Dataset):
    def __init__(self, num_samples, transform=None, PCA = None):
        self.num_samples = num_samples
        self.transform = transform
        self.PCA = PCA
        self.data, self.output = self.generate_data()

    def generate_data(self):
        data = []
        output_concat = []
        for _ in range(self.num_samples):
            # Generate random image data (3 channels, 128x128 pixels)
            image = np.random.randint(0, 256, size=(3, 128, 128), dtype=np.uint8)
            image = Image.fromarray(np.transpose(image, (1, 2, 0)))

            # Generate random output array of length 1000
            output = np.random.rand(1000)

            data.append((image, output))
            output_concat.append(output)
        if self.PCA: self.PCA.fit(output_concat)
        return data, output_concat
    
    def give_output(self):
        return self.output
    
    def internal_PCA(self):
        if self.PCA:
            return self.PCA
        else:
            print(f'PCA is {self.PCA}')

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image, output = self.data[idx]

        if self.transform:
            image = self.transform(image)
        if self.PCA:
            output = self.PCA.transform(output.reshape(1, -1))
        return image, torch.FloatTensor(output[0])

# Define data transformations (you can customize these as needed)
data_transform = transforms.Compose([
    transforms.ToTensor(),
])


# Create an instance of the ImaginaryDataset
num_samples = 1000  # You can adjust this based on your needs
imaginary_dataset = ImaginaryDataset(num_samples, transform= data_transform, PCA = PCA(n_components= 100))


# Create a DataLoader for batch processing
batch_size = 32
data_loader = DataLoader(imaginary_dataset, batch_size=batch_size, shuffle=True)

In [69]:
class ImaginaryDataset(Dataset):
    def __init__(self, images_list, outputs_list, transform=None, PCA=None):
        self.num_samples = len(images_list)
        self.transform = transform
        self.PCA = PCA
        self.data, self.output = self.load_data(images_list, outputs_list)

    def load_data(self, images_list, outputs_list):
        data = []
        output_concat = []

        for i in range(self.num_samples):
            # Load image from the given list
            image = Image.fromarray(images_list[i])

            # Load output array from the given list
            output = outputs_list[i]

            data.append((image, output))
            output_concat.append(output)

        if self.PCA:
            self.PCA.fit(output_concat)

        return data, output_concat

    def give_output(self):
        return self.output

    def internal_PCA(self):
        if self.PCA:
            return self.PCA
        else:
            print(f'PCA is {self.PCA}')

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image, output = self.data[idx]

        if self.transform:
            image = self.transform(image)
        if self.PCA:
            output = self.PCA.transform(output.reshape(1, -1))

        return image, torch.FloatTensor(output[0])

In [68]:
Frozen_PCA = imaginary_dataset.internal_PCA()

PCA is None


In [65]:
len(Frozen_PCA.inverse_transform(imaginary_dataset[0][1]))

1000

In [72]:
from ..utils import yay

ImportError: attempted relative import with no known parent package

In [24]:
class Trainer:
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.loss_fn = None
        self.train_loader = None
        self.val_loader = None

    def compile(self, model, optimizer, learning_rate, loss_fn):
        self.model = model
        self.optimizer = optimizer(self.model.parameters(), lr=learning_rate)
        self.loss_fn = loss_fn

    def fit(self, num_epochs, train_loader, val_loader = None):
        self.train_loader = train_loader
        self.val_loader = val_loader

        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0.0
            for inputs, targets in self.train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(self.train_loader)
            print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_loss}")

            if self.val_loader is not None:
                val_loss = self.evaluate(self.val_loader, "Validation")
                print(f"Validation Loss: {val_loss}")

    def evaluate(self, data_loader, mode="Test"):
        self.model.eval()
        total_loss = 0.0
        with torch.no_grad():
            for inputs, targets in data_loader:
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)
                total_loss += loss.item()

        avg_loss = total_loss / len(data_loader)
        print(f"{mode} Loss: {avg_loss}")
        return avg_loss

In [25]:
trainer = Trainer()
trainer.compile(model = model, optimizer= torch.optim.Adam, learning_rate= 0.0001, loss_fn= torch.nn.MSELoss())
trainer.fit(train_loader= data_loader, num_epochs= 5)

Epoch [1/5], Training Loss: 0.46090528555214405
Epoch [2/5], Training Loss: 0.25140242744237185
Epoch [3/5], Training Loss: 0.2060776068829
Epoch [4/5], Training Loss: 0.1721035768277943
Epoch [5/5], Training Loss: 0.13985892571508884


In [14]:
trainer.fit(10)

Epoch [1/10], Loss: 0.47111201379448175
Epoch [2/10], Loss: 0.2546487138606608
Epoch [3/10], Loss: 0.20916434470564127
Epoch [4/10], Loss: 0.1749093383550644
Epoch [5/10], Loss: 0.14236912317574024
Epoch [6/10], Loss: 0.1145813858602196
Epoch [7/10], Loss: 0.09303548978641629
Epoch [8/10], Loss: 0.07836320041678846
Epoch [9/10], Loss: 0.06674484279938042
Epoch [10/10], Loss: 0.058404486393556
